# Practice Session 04: Basket analysis

Author: Tània Pazos Puig

E-mail: tania.pazos01@inssobreques.upf.edu

Date: 16/10/2024

In [1]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import csv
import gzip

from apyori import apriori

# 1. Playing with apyori

The function below prints the output of the apyori library in a readable format. We will later use it to print the results of our association rules mining.

In [2]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]
                
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

We now define a set of 23 transactions representing supermarket baskets.

In [3]:
transactions = [
    ['Almond Milk', 'Tofu', 'Olive Oil'],
    ['Beer', 'Crisps', 'Olives'], 
    ['Almond Milk', 'Rice Milk', 'Olive Oil'],
    ['Almond Milk', 'Tofu', 'Tomatoes', 'Banana', 'Olive Oil'],
    ['Bread', 'Butter', 'Jam'], 
    ['Jam', 'Eggs', 'Bread', 'Butter'], 
    ['Eggs', 'Cheese'],
    ['Rice Milk', 'Almond Milk', 'Tofu', 'Strawberries'],
    ['Straweberries', 'Sugar', 'Cream'],
    ['Bread', 'Olive Oil', 'Almond MIlk', 'Tofu'],
    ['Beer', 'Olives', 'Cheese', 'Crisps', 'Bread'], 
    ['Eggs', 'Butter', 'Cheese', 'Tomatoes'],
    ['Pickles', 'Cheese', 'Meat'],
    ['Meat', 'Bread', 'Pickles'],
    ['Mayo', 'Bread', 'Pickles', 'Crisps', 'Beer', 'Olives'], 
    ['Jam', 'Biscuits', 'Butter', 'Bread'], 
    ['Tofu', 'Blueberries', 'Almond Milk', 'Olive Oil'],
    ['Pickles', 'Eggs', 'Butter', 'Meat'],
    ['Crisps', 'Mayo', 'Bread', 'Cheese'],
    ['Strawberries', 'Cream', 'Tofu', 'Almond Milk'],
    ['Crisps', 'Olives', 'Beer', 'Mayo'], 
    ['Tofu', 'Olive Oil', 'Almond Milk', 'Bread', 'Pickles'],
    ['Eggs', 'Cheese', 'Pickles', 'Crisps', 'Meat'],
]

Now we will printout all the rules we have obtained, and for two particular rules (['Beer', 'Crisps'] => ['Olives'] and ['Jam', 'Butter'] => ['Bread']) indicate how the support, confidence, and lift is calculated.

In [4]:
results = list(apriori(transactions, min_support=0.1, min_confidence=0.9, min_lift=1.0))
print_apyori_output(results)

Rules involving itemset ['Beer', 'Crisps']
['Beer'] => ['Crisps'] (support=0.1739, confidence=1.00, lift=3.83)

Rules involving itemset ['Beer', 'Olives']
['Beer'] => ['Olives'] (support=0.1739, confidence=1.00, lift=5.75)
['Olives'] => ['Beer'] (support=0.1739, confidence=1.00, lift=5.75)

Rules involving itemset ['Bread', 'Jam']
['Jam'] => ['Bread'] (support=0.1304, confidence=1.00, lift=2.56)

Rules involving itemset ['Butter', 'Jam']
['Jam'] => ['Butter'] (support=0.1304, confidence=1.00, lift=4.60)

Rules involving itemset ['Crisps', 'Mayo']
['Mayo'] => ['Crisps'] (support=0.1304, confidence=1.00, lift=3.83)

Rules involving itemset ['Crisps', 'Olives']
['Olives'] => ['Crisps'] (support=0.1739, confidence=1.00, lift=3.83)

Rules involving itemset ['Meat', 'Pickles']
['Meat'] => ['Pickles'] (support=0.1739, confidence=1.00, lift=3.83)

Rules involving itemset ['Beer', 'Crisps', 'Olives']
['Beer'] => ['Crisps', 'Olives'] (support=0.1739, confidence=1.00, lift=5.75)
['Olives'] => ['B

To compute the support of ['Beer', 'Crisps'] => ['Olives'], we count the number of transactions where 'Beer', 'Crisps', and 'Olives' appear together and divide by the total number of transactions. Hence, support(['Beer', 'Crisps'] => ['Olives']) = 4/23 = 0.1739. For the confidence, we divide the computed support over the support of the antecedent ['Beer', 'Crisps']. Since support(['Beer', 'Crisps']) = 0.1739, confidence(['Beer', 'Crisps'] => ['Olives']) = 0.1739/0.1739 = 1. Finally, to compute the lift of the rule we divide the support of the rule over the support of ['Beer', 'Crisps'] (0.1739) times the support of ['Olives'] (4/23 = 0.1439). Hence, lift(['Beer', 'Crisps'] => ['Olives']) = 0.1739 / (0.1739 x 0.1739) = 5.75. <p>
Similarly, support(['Jam', 'Butter'] => ['Bread']) = 3/23 = 0.1304, confidence(['Jam', 'Butter'] => ['Bread']) = 0.1304 / 0.1304 = 1, and lift(['Jam', 'Butter'] => ['Bread']) = 0.1304 / (0.1304 x 0.3913) = 2.56.

# 2. Load and prepare the shopping baskets

The following code loads the information about products into a dataframe indexed by product id.

In [5]:
# LEAVE AS-IS

# File names
INPUT_PRODUCTS = "data/instacart/instacart-products.csv"
INPUT_TRANSACTIONS = "data/instacart/instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

As this file is large and complex, we will focus on one or two departments and try to get some conclusions about the products in those departments. The following code lists some department names.

In [6]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

The function below `select_from_departments` takes as input a dataframe containing product information, a list of product ids, and a list of department ids. It returns a list containing only the product ids that belong to one of the listed departments. 

In [7]:
def select_from_departments(products, product_ids, department_ids):
    selected_prods = []
    for product_id in product_ids:
        if products.loc[product_id].department_id in department_ids:
            selected_prods.append(product_id)
    return selected_prods

We now test our function by passing it a list of products and ensuring it selects only the products in the 1, 2, or 3 departments we specify. Each test case will print:

* The product name and department id of each item in the input list.
* The product name and department id of each item in the output list, i.e. the selected products list.

In [8]:
def print_test_case(products, product_ids, selected_products):
    print("\nTest case:")
    print(product_ids)
    
    print("\nInput products:")
    for product_id in product_ids:
        product_name = products.loc[product_id].product_name
        department_id = products.loc[product_id].department_id
        print(f"{product_id} {product_name} (dept {department_id})")
    
    print("\nSelected products:")
    for product_id in selected_products:
        product_name = products.loc[product_id].product_name
        department_id = products.loc[product_id].department_id
        print(f"{product_id} {product_name} (dept {department_id})")

In [9]:
test_case1 = [22, 26, 45, 54, 57, 71, 111, 112]
departments_to_select1 = [DEPT_BAKERY, DEPT_CLEANING]
selected_prods1 = select_from_departments(products, test_case1, departments_to_select1)

test_case2 = [35, 216, 599, 830, 40, 99, 613, 980]
departments_to_select2 = [DEPT_PETS, DEPT_PHARMACY, DEPT_BABIES]
selected_prods2 = select_from_departments(products, test_case2, departments_to_select2)

test_case3 = [489, 312, 25, 742, 514, 101, 206, 895]
departments_to_select3 = [DEPT_VEGGIES, DEPT_ALCOHOL]
selected_prods3 = select_from_departments(products, test_case3, departments_to_select3)

print_test_case(products, test_case1, selected_prods1)
print_test_case(products, test_case2, selected_prods2)
print_test_case(products, test_case3, selected_prods3)


Test case:
[22, 26, 45, 54, 57, 71, 111, 112]

Input products:
22 Fresh Breath Oral Rinse Mild Mint (dept 11)
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8)
45 European Cucumber (dept 4)
54 24/7 Performance Cat Litter (dept 8)
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)
112 Hot Tomatillo Salsa (dept 13)

Selected products:
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)

Test case:
[35, 216, 599, 830, 40, 99, 613, 980]

Input products:
35 Italian Herb Porcini Mushrooms Chicken Sausage (dept 12)
216 Gluten Free Oatmeal, Variety Pack (dept 14)
599 Floor Care Tile & Vinyl Floor Finish Clean Scent (dept 17)
830 Skinny Wheat Buns (dept 3)
40 Beef Hot Links Beef Smoked Sausage With Chile Peppers (dept 12)
99 Local Living Butter Lettuce (dept 4)
613 Infinity Always Infinity Size 2 Super Pads with Wings, Unscen

## 2.2. Read and filter transactions

The code below reads the transactions file and keeps only those transactions with at least one item in the specified target departments. These transactions are stored into an array named `transactions`. We stop the code after storing 5000 of the transactions read. Every 1000 transactions read, we print the number of transactions read and the number of transactions stored.

In [10]:
def extract_transactions_by_departments(target_departments, max_transactions = 5000):

    transactions = []
    transactions_read = 0 # Count number of transactions read

    # Open the compressed input file
    with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:

        # Create a CSV reader
        reader = csv.reader(inputfile, delimiter=",")

        # Iterate through the CSV file
        for row in reader:

            transactions_read += 1

            # Convert to integers
            items = [int(x) for x in row]

            filtered_items = [item for item in items if products.loc[item].department_id in target_departments]

            # If there are filtered items, store the transaction
            if filtered_items:
                transactions.append(filtered_items)

            # Print status every 1000 transactions read
            if transactions_read % 1000 == 0:
                print(f"Transactions read: {transactions_read}, Transactions stored: {len(transactions)}")

            # Stop reading when 5000 transactions stored
            if len(transactions) >= 5000:
                break
 
    return transactions

To test our function, we filter out those transactions that do not contain any item in `DEPT_PHARMACY`.

In [11]:
target_departments = [DEPT_PHARMACY]
transactions_pharmacy = extract_transactions_by_departments(target_departments)

Transactions read: 1000, Transactions stored: 102
Transactions read: 2000, Transactions stored: 219
Transactions read: 3000, Transactions stored: 313
Transactions read: 4000, Transactions stored: 422
Transactions read: 5000, Transactions stored: 526
Transactions read: 6000, Transactions stored: 619
Transactions read: 7000, Transactions stored: 722
Transactions read: 8000, Transactions stored: 810
Transactions read: 9000, Transactions stored: 920
Transactions read: 10000, Transactions stored: 1046
Transactions read: 11000, Transactions stored: 1152
Transactions read: 12000, Transactions stored: 1245
Transactions read: 13000, Transactions stored: 1352
Transactions read: 14000, Transactions stored: 1459
Transactions read: 15000, Transactions stored: 1559
Transactions read: 16000, Transactions stored: 1663
Transactions read: 17000, Transactions stored: 1753
Transactions read: 18000, Transactions stored: 1854
Transactions read: 19000, Transactions stored: 1943
Transactions read: 20000, Tran

And check that each transaction in `transactions_pharmacy` contains at least one item belonging to `DEPT_PHARMACY` (11).

In [12]:
for transaction in transactions_pharmacy[:5]:
    print([products.loc[item, 'department_id'] for item in transaction])

[11]
[11]
[11, 11]
[11]
[11]


## 2.3. Extract association rules and comment on them

Now, we will run the association rules mining algorithm over the selected transactions.

In [13]:
# Set parameters for the apriori algorithm
min_support = 0.001
min_confidence = 0.1
min_lift = 5

results = list(apriori(transactions_pharmacy, min_support=min_support, min_confidence=min_confidence, min_lift=min_lift))

print_apyori_output(results, products, 'product_name')

Rules involving itemset [16825, 1406]
['Vegan Nutritional Shake Sweet Vanilla Bean'] => ['Vegan Smooth Chocolate Nutritional Shake'] (support=0.0012, confidence=0.40, lift=80.00)
['Vegan Smooth Chocolate Nutritional Shake'] => ['Vegan Nutritional Shake Sweet Vanilla Bean'] (support=0.0012, confidence=0.24, lift=80.00)

Rules involving itemset [7976, 16019]
['One Plant-Based Chocolate Flavor Nutritional Shake Drink Mix'] => ['All-In-One French Vanilla Nutritional Shake Sachet'] (support=0.0010, confidence=0.31, lift=97.66)
['All-In-One French Vanilla Nutritional Shake Sachet'] => ['One Plant-Based Chocolate Flavor Nutritional Shake Drink Mix'] (support=0.0010, confidence=0.31, lift=97.66)

Rules involving itemset [27544, 39162]
['Lemon Verbena Hand Soap'] => ['Clean Day Basil Hand Soap'] (support=0.0018, confidence=0.12, lift=11.21)
['Clean Day Basil Hand Soap'] => ['Lemon Verbena Hand Soap'] (support=0.0018, confidence=0.16, lift=11.21)



Firstly, rules involving itemset [16825, 1406] indicate that customers buying 'Vegan Nutritional Shake Sweet Vanilla Bean' are likely to also buy 'Vegan Smooth Chocolate Nutritional Shake' and viceversa. This itemset is present in 0.12% of all transactions. What is more, 40% of customers who buy Sweet Vanilla Bean shake also buy Smooth Chocolate Shake. Hence, the app could display the Vegan Smooth Chocolate Nutritional Shake to customers who have added the Sweet Vanilla Bean flavor to their cart. The value of 80.00 for the lift shows that there is a very high likelihood that the two shakes are bought together. <p> Similarly, rules regarding itemset [7976, 16019] show that if customers buy One Plant-Based Chocolate Flavor Nutritional Shake Drink Mix they are likely to buy All-In-One French Vanilla Nutritional Shake Sachet and viceversa. Although the support and confidence do not significantly differ from the previous example, the high lift of the rule (97.66) suggests that implementing an offer discount for purchasing both items together could be an effective strategy. <p> Finally, rules involving hand soaps (itemset [27544, 39162]) indicate that 12% of customers who buy Lemon Verbena Hand Soap also buy Clean Day Basil Hand Soap. However, the lift of the rule suggests there is a moderate relationship between the hand soaps. To increase sales of both items, the shopping app could make the client receive a discount for the Clean Day Basil Soap after buying the Lemon Verbena.

## 2.4. Extract association rules and comment on them (other departments)

The code below selects departments `DEPT_BAKERY` and `DEPT_VEGGIES` and extract transactions again. 

In [14]:
target_departments = [DEPT_BAKERY, DEPT_VEGGIES]
transactions_bakery_veggies = extract_transactions_by_departments(target_departments)

Transactions read: 1000, Transactions stored: 817
Transactions read: 2000, Transactions stored: 1600
Transactions read: 3000, Transactions stored: 2400
Transactions read: 4000, Transactions stored: 3193
Transactions read: 5000, Transactions stored: 3971
Transactions read: 6000, Transactions stored: 4749


We check that items in the transactions stored are from the specified departments.

In [15]:
for transaction in transactions_bakery_veggies[:5]:
    print([products.loc[item, 'department_id'] for item in transaction])

[4, 4, 4]
[4, 4, 4, 3]
[3]
[4, 4, 4, 4, 4, 4, 4]
[3]


And we obtain the association rules.

In [16]:
# Set parameters for the apriori algorithm
min_support = 0.005
min_confidence = 0.1
min_lift = 4

results2 = list(apriori(transactions_bakery_veggies, min_support=min_support, min_confidence=min_confidence, min_lift=min_lift))

print_apyori_output(results2, products, 'product_name')

Rules involving itemset [43352, 16797]
['Raspberries'] => ['Strawberries'] (support=0.0050, confidence=0.24, lift=4.24)

Rules involving itemset [24964, 22935]
['Organic Yellow Onion'] => ['Organic Garlic'] (support=0.0082, confidence=0.18, lift=4.03)
['Organic Garlic'] => ['Organic Yellow Onion'] (support=0.0082, confidence=0.18, lift=4.03)

Rules involving itemset [26209, 31717]
['Limes'] => ['Organic Cilantro'] (support=0.0068, confidence=0.14, lift=5.34)
['Organic Cilantro'] => ['Limes'] (support=0.0068, confidence=0.25, lift=5.34)

Rules involving itemset [13176, 47209, 27966]
['Organic Raspberries'] => ['Bag of Organic Bananas', 'Organic Hass Avocado'] (support=0.0056, confidence=0.10, lift=4.07)
['Bag of Organic Bananas', 'Organic Hass Avocado'] => ['Organic Raspberries'] (support=0.0056, confidence=0.22, lift=4.07)



The extracted association rules on `DEPT_FOOD` and `DEPT_VEGGIES` suggest strong relationships between several items. <p> Firstly, customers who purchase raspberries are likely to purchase strawberries. Indeed, the moderately high value of the lift (4.24) indicates that the two products are strongly correlated. Hence, it would be advisable to suggest strawberries when a client adds raspberries to their cart. <p> Secondly, we can observe a moderate association between organic yellow onions and organic garlics. This can be explained because onions and garlic appear together in many recipes. What is more, customers purchasing some organic product usually show preference for buying multiple organic items, rather than just a single product. This is the case of the last association rule, which shows that clients purchasing organic bananas and organic hass avocado are likely to also purchase organic raspberries. <p> The associaton between limes and organic cilantro suggests that customers generally buy them together as they are key ingredients in Mexican cuisine. Therefore, it would be a good idea to display limes in the shopping app right after customers have searched organic cilantro. <p> It should be noted that there is a lack of strong associations for bakery products compared to vegetables. This suggests that bakery items are bought more independently or not as frequently in combination with other bakery or vegetable items. Hence, the shopping app should focus more on personalized vegetable suggestions. It is also worth highlighting that, despite having the same number of transactions (5000) for both pharmacy and veggies-bakery departments, we were able to increase the minimum support threshold from 0.001 to 0.005 for veggies-bakery while still obtaining a similar number of association rules. This indicates that items from the veggie-bakery department (particularly from the veggie department) have more frequent item combinations than pharmacy items. However, the decrease in the lift value for veggies-bakery rules compared to pharmacy items indicates that pharmacy products are more likely to be co-purchased.

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>